# Homework 4
## Author: Siddhant Madan

### Import libraries

In [1]:
import warnings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
warnings.filterwarnings("ignore")

In [2]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os

### Fetch the dataset

We need the dataset here to find out the available dates and tokens in the dataset

In [3]:
connection_string = 'sqlite:///data/data.db'

prices = pd.read_sql(f'''
SELECT 
    * 
FROM ohlc 
''', connection_string)
prices['ts'] = pd.to_datetime(prices['ts'])

prices.set_index('ts', inplace=True)

prices.head()

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-11-01 00:00:00,61421.37,61669.14,61239.60,61343.68,256.433869,1.575751e+07,BTC,BTC
2021-11-01 01:00:00,61346.17,61709.82,61171.22,61610.93,332.481185,2.044558e+07,BTC,BTC
2021-11-01 02:00:00,61610.94,61779.87,61299.89,61333.17,314.250720,1.935390e+07,BTC,BTC
2021-11-01 03:00:00,61333.17,61457.28,60050.00,60589.06,1059.931358,6.414625e+07,BTC,BTC
2021-11-01 04:00:00,60590.23,60655.00,59752.92,59971.89,621.419878,3.744744e+07,BTC,BTC


### Get the dates for the analysis period

In [4]:
def last_dow(d: datetime, dow: int) -> datetime:
    '''
    From the 7 days gone by before `d`, get the date specified by `dow` 
    (using datetime convention of Monday is 0 and Sunday is 6)
    
    Basically, use this function if you want to find out what was the
    date on the last Monday or the last Tuesday or whatever
    
    '''
    dow_d = d.weekday()
    days_delta = (dow_d - dow + 7) if dow_d < dow else dow_d - dow
    return d - timedelta(days=days_delta)

To get the last two complete weeks (i.e. two full periods of Monday to Sunday) we will first get the last Sunday available in the dataset, and then using this as the last date of our analysis, we will get the two previous rolling weeks.

Note: We will send the Monday after this last Sunday to the analysis notebook even though this date may not be in the dataset; this is because in the analysis notebook, the `to_date` argument is exclusive.

In [5]:
last_date = prices.index.sort_values()[-1] # last available date in the dataset
last_sunday = last_dow(last_date, 6) # last Sunday in the dataset
end_date = last_sunday + timedelta(days=1) # this is the Monday after the last available Sunday
start_date = end_date - timedelta(days=14) # two Mondays before the end_date

print(f'Our analysis will run from {start_date} (inclusive) to {end_date} (exclusive)')

Our analysis will run from 2021-11-29 00:00:00 (inclusive) to 2021-12-13 00:00:00 (exclusive)


### Get all the unique tokens in the dataset

In [6]:
tokens = prices.token.unique()

In [7]:
print(f'Tokens present in the dataset: {tokens}')

Tokens present in the dataset: ['BTC' 'ETH' 'USDT' 'SOL' 'ADA' 'DOT' 'AVAX' 'ATOM' 'CRV' 'AAVE' 'COMP']


### Get path to output folder

Create the folder if not present already

In [8]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

### Run the template for all the tokens

In [9]:
for token in tokens:
    print(f'running for {token} for date {start_date} to {end_date}', end='\r')
    res = pm.execute_notebook(
        'hw_4_Siddhant_Madan_template.ipynb',
        f'{base_output_folder}/hw_4_Siddhant_Madan_{token}_{start_date}.ipynb',
        parameters = {
            'from_date': f'{start_date}',
            'to_date': f'{end_date}',
            'token': token,
            'connection_string': connection_string,
        }
    )

Executing:   0%|          | 0/22 [00:00<?, ?cell/s]

Executing:   0%|          | 0/22 [00:00<?, ?cell/s]

Executing:   0%|          | 0/22 [00:00<?, ?cell/s]

Executing:   0%|          | 0/22 [00:00<?, ?cell/s]

Executing:   0%|          | 0/22 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/22 [00:00<?, ?cell/s]

Executing:   0%|          | 0/22 [00:00<?, ?cell/s]

Executing:   0%|          | 0/22 [00:00<?, ?cell/s]

Executing:   0%|          | 0/22 [00:00<?, ?cell/s]

Executing:   0%|          | 0/22 [00:00<?, ?cell/s]

### Read the created notebooks and show summary statistics

In [10]:
nbs = sb.read_notebooks('outputs')

In [11]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'avg_hourly_return': scrap['prices'].data.returns.mean(),
        'avg_volatility': scrap['prices'].data.volatility.mean(),
        'max_drawdown': scrap['prices'].data.max_drawdown.min(),
        'price_range': (scrap['prices'].data.close[-1] - scrap['prices'].data.close[0])
    }) 
    for book, scrap in nbs.notebook_scraps.items()
])

,token,from_date,to_date,avg_hourly_return,avg_volatility,max_drawdown,price_range
0,AAVE,2021-11-29 00:00:00,2021-12-13 00:00:00,-7.697463e-04,0.212356,-0.378718,-58.6450
1,ADA,2021-11-29 00:00:00,2021-12-13 00:00:00,-4.241618e-04,0.217477,-0.312827,-0.2445
2,ATOM,2021-11-29 00:00:00,2021-12-13 00:00:00,-1.776982e-04,0.299425,-0.329570,-2.7560
3,AVAX,2021-11-29 00:00:00,2021-12-13 00:00:00,-5.540979e-04,0.286536,-0.385790,-22.9300
4,BTC,2021-11-29 00:00:00,2021-12-13 00:00:00,-3.569235e-04,0.154894,-0.206462,-6999.8400
5,COMP,2021-11-29 00:00:00,2021-12-13 00:00:00,-1.015116e-03,0.224354,-0.377871,-88.2800
6,CRV,2021-11-29 00:00:00,2021-12-13 00:00:00,-5.385247e-04,0.328177,-0.394862,-1.0362
7,DOT,2021-11-29 00:00:00,2021-12-13 00:00:00,-4.540131e-04,0.245633,-0.350800,-6.0050
8,ETH,2021-11-29 00:00:00,2021-12-13 00:00:00,-6.065377e-05,0.174104,-0.188101,-141.7100
9,SOL,2021-11-29 00:00:00,2021-12-13 00:00:00,-3.446384e-04,0.242432,-0.317207,-26.6870
